In [31]:
import json
import os
from tqdm import tqdm
from multiprocessing import Pool
import sys
import time
sys.path.append('/home/rahul/ripper/GitRipper')

import pandas as pd
from GitRipper.Ripper import collect

In [32]:
cpu_count = os.cpu_count()
print(cpu_count)

8


In [33]:
columns = ['owner', 'repo', 'name', 'description', 'licenseName', 'licenseSpdxId', 'licenseUrl',
           'shortDescriptionHTML', 'repourl', 'createdAt', 'updatedAt', 'pushedAt',
           'forkCount', 'stargazerCount', 'issuesCount', 'pullRequestsCount']

In [34]:
parallelcount = cpu_count

In [35]:
#Load the json file
with open('/home/rahul/ripper/GitRipper/API_keys.json') as f:
    keys = json.load(f)['keys']

In [36]:
#create an empty dataframe and save as csv
if not os.path.exists('/home/rahul/oss/code/Codegov_Repos.csv'):
    df = pd.DataFrame(columns=columns)
    df.to_csv('/home/rahul/oss/code/Codegov_Repos.csv', index=False)

In [37]:
#get all the packages alrady collected
slugs_collected = []
for df in pd.read_csv('/home/rahul/oss/code/Codegov_Repos.csv', chunksize=1000):
    repo_names = df['repo'].tolist()
    repo_owners = df['owner'].tolist()
    for i in range(len(repo_names)):
        slugs_collected.append(repo_owners[i] + '/' + repo_names[i])
    slugs_collected = [i.strip() for i in list(set(slugs_collected))]

In [38]:
def get_owner_and_repo(url):
    ### if it has .git, remove it
    if url.endswith(".git"):
        url = url[:-4]
    ### get owner and repo
    owner = url.split("/")[-2]
    repo = url.split("/")[-1]
    return owner, repo

In [39]:
keys_list = [(i.split(":")[0], i.split(":")[1]) for i in keys]
collector = collect(keys_list)

In [40]:
?collector.getRepoInfo

Signature: collector.getRepoInfo(owner, repo, token=None)
Docstring:
This function collects all commits from a repo since a given date and returns a dataframe
owner: owner of the repo
repo: name of the repo
File:      ~/ripper/GitRipper/GitRipper/Ripper.py
Type:      method

In [41]:
def get_repo_info(owner, repo, key=None):
    try:
        if owner + '/' + repo in slugs_collected:
            return None
        repo_info = collector.getRepoInfo(owner, repo, key)
        del repo_info['readme']
        return repo_info
    except Exception as e:
        print("Error: ", e)
        return None

In [42]:
def get_owner_and_repo(url):
    ### if it has .git, remove it
    if url.endswith(".git"):
        url = url[:-4]
    ### get owner and repo
    owner = url.split("/")[-2]
    repo = url.split("/")[-1]
    return owner, repo

In [43]:
total_pacakges = pd.read_csv("/home/rahul/oss/code/code_repos.csv").shape[0]

In [44]:
for df in tqdm(pd.read_csv("/home/rahul/oss/code/code_repos.csv", chunksize=parallelcount), total=total_pacakges//parallelcount):
    try:
        slugs = [get_owner_and_repo(i) for i in df.RepoURL]
        owners = [i[0] for i in slugs]
        repos = [i[1] for i in slugs]
        owners = [i[0] for i in slugs]
        repos = [i[1] for i in slugs]
        best_keys = collector.getBestKeys(n=parallelcount)
        args = list(zip(owners, repos, best_keys))
    except:
        continue
    with Pool(parallelcount) as p:
        result = p.starmap(get_repo_info, args)
    result = [i for i in result if i is not None]
    if len(result) == 0:
        continue
    result_df = pd.DataFrame.from_dict(result, orient='columns')
    result_df.to_csv('/home/rahul/oss/code/Codegov_Repos.csv', mode='a', header=False, index=False)

  0%|          | 0/759 [00:00<?, ?it/s]

Error:  'NoneType' object is not subscriptable


  0%|          | 1/759 [00:00<10:16,  1.23it/s]

Error:  'NoneType' object is not subscriptable


  0%|          | 2/759 [00:01<08:43,  1.45it/s]

...

Error:  'NoneType' object is not subscriptable


 17%|█▋        | 130/759 [01:25<07:55,  1.32it/s]

Error:  'NoneType' object is not subscriptable
Error:  'NoneType' object is not subscriptable


 17%|█▋        | 130/759 [01:26<06:56,  1.51it/s]


KeyboardInterrupt: 